## GPU 메모리 분해

In [1]:
import torch

def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU 메모리 사용량:  {used_memory:.3f} GB")
    else:
        print("런타임 유형을 GPU로 변경하세요.")

print_gpu_utilization()
# GPU 메모리 사용량:  0.000 GB

GPU 메모리 사용량:  0.000 GB


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

def load_model_and_tokenizer(model_name, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map={"": 0})
    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map={"": 0})
        lora_config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=['query_key_value'],
            lora_dropout=0.05,
            bias='none',
            task_type='CAUSAL_LM',
        )
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
    elif peft == 'qlora':
        lora_config = LoraConfig(
            r=8,
            lora_alpha=32,
            target_modules=['query_key_value'],
            lora_dropout=0.05,
            bias='none',
            task_type='CAUSAL_LM'
        )
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.float16
        )
        model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"": 0})
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

model_name = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_name)
print("모델 파라미터 데이터 타입: ", model.dtype)
# GPU 메모리 사용량:  2.599 GB
# 모델 파라미터 데이터 타입:  torch.float16

/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량:  2.599 GB
모델 파라미터 데이터 타입:  torch.float16


In [3]:
from transformers import AdamW
from torch.utils.data import DataLoader

def estimate_memory_of_gradients(model):
    total_memory = 0
    for param in model.parameters():
        if param.grad is not None:
            total_memory += param.grad.nelement() * param.grad.element_size()
    return total_memory

def estimate_memory_of_optimizer(optimizer):
    total_memory = 0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                total_memory += v.nelement() * v.element_size()
    return total_memory

In [4]:
def train_model(model, dataset, training_args):
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()
    
    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()
    gpu_utilization_printed = False

    gradients_memory = 0
    optimizer_memory = 0 

    for step, batch in enumerate(train_dataloader, start=1):
        batch = {k: v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / training_args.gradient_accumulation_steps
        loss.backward()

        if step % training_args.gradient_accumulation_steps == 0:
            optimizer.step()
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)
            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True
            optimizer.zero_grad()
        
        print(f"그레이디언트 메모리 사용량: {gradients_memory / (1024 ** 3):.3f} GB")
        print(f"옵티마이저 상태의 메모리 사용량: {optimizer_memory / (1024 ** 3):.3f} GB")

In [5]:
import numpy as np
from datasets import Dataset

def make_dummy_dataset():
    seq_len, dataset_size = 256, 64
    dummy_data = {
        'input_ids': np.random.randint(100, 30000, (dataset_size, seq_len)),
        'labels': np.random.randint(100, 30000, (dataset_size, seq_len))
    }
    dataset = Dataset.from_dict(dummy_data)
    dataset.set_format('pt')
    return dataset

In [6]:
import gc 

def cleanup():
    if 'model' in globals():
        del globals()['model']
    if 'dataset' in globals():
        del globals()['dataset']
    gc.collect()
    torch.cuda.empty_cache()

In [7]:
from transformers import TrainingArguments, Trainer

def gpu_memory_experiment(
    batch_size,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    model_name='EleutherAI/polyglot-ko-1.3b',
    peft=None
):
    print(f"배치 크기: {batch_size}")
    model, tokenizer = load_model_and_tokenizer(model_name, peft=peft)
    if gradient_checkpointing == True or peft == 'qlora':
        model.config.use_cache = False
    
    dataset = make_dummy_dataset()

    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        gradient_checkpointing=gradient_checkpointing,
        output_dir='../results',
        num_train_epochs=1
    )

    try:
        train_model(model, dataset, training_args)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally:
        del model, dataset
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()

2024-11-12 15:01:01.696966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 15:01:01.697021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 15:01:01.697832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 15:01:01.702162: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-12 15:01:04.323669: W tensorflow/compiler/tf2

In [8]:
cleanup()
print_gpu_utilization()

GPU 메모리 사용량:  0.000 GB


In [9]:
for batch_size in [4, 8, 16]:
    gpu_memory_experiment(batch_size)

    torch.cuda.empty_cache()

배치 크기: 4


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량:  2.599 GB


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량:  2.615 GB


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GPU 메모리 사용량:  12.164 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
GPU 메모리 사용량:  0.016 GB


## 그레이디언트 누적

In [18]:
cleanup()
print_gpu_utilization()

GPU 메모리 사용량:  5.623 GB


In [9]:
gpu_memory_experiment(batch_size=4, gradient_accumulation_steps=4)

torch.cuda.empty_cache()

배치 크기: 4


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량:  2.599 GB


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


그레이디언트 메모리 사용량: 0.000 GB
옵티마이저 상태의 메모리 사용량: 0.000 GB
그레이디언트 메모리 사용량: 0.000 GB
옵티마이저 상태의 메모리 사용량: 0.000 GB
그레이디언트 메모리 사용량: 0.000 GB
옵티마이저 상태의 메모리 사용량: 0.000 GB
GPU 메모리 사용량:  10.586 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
GPU 메모리 사용량:  0.016 GB


## 그레이디언트 체크포인팅

In [ ]:
cleanup()
print_gpu_utilization()

In [10]:
gpu_memory_experiment(batch_size=16, gradient_checkpointing=True)

배치 크기: 16


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량:  2.615 GB


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


GPU 메모리 사용량:  10.290 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레이디언트 메모리 사용량: 2.481 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
GPU 메모리 사용량:  0.016 GB


## LoRA 학습법

In [11]:
cleanup()
print_gpu_utilization()

GPU 메모리 사용량:  0.016 GB


In [12]:
gpu_memory_experiment(batch_size=16, peft='lora')
torch.cuda.empty_cache()

배치 크기: 16


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.1180
GPU 메모리 사용량:  2.621 GB


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GPU 메모리 사용량:  4.744 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
GPU 메모리 사용량:  0.016 GB


## QLoRA 학습법

In [13]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='qlora')

GPU 메모리 사용량:  0.945 GB
배치 크기: 16


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.1180
GPU 메모리 사용량:  2.112 GB


/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/woosungkim/anaconda3/envs/machine_learning/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.aut

GPU 메모리 사용량:  2.651 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레이디언트 메모리 사용량: 0.006 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
GPU 메모리 사용량:  0.945 GB
